# 1. Импорт библиотек и настройка

In [26]:
import pandas as pd
import numpy as np

DATA_PATH = 'C:/Users/kokontsev/Desktop/Sleep-Quality-Prediction/data/raw/'
OUTPUT_PATH = "C:/Users/kokontsev/Desktop/Sleep-Quality-Prediction/data/processed/"

pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 120)

# 2. Загрузка данных

In [9]:
daily_activity = pd.read_csv(DATA_PATH + "dailyActivity_merged.csv")
daily_steps = pd.read_csv(DATA_PATH + "dailySteps_merged.csv")
daily_calories = pd.read_csv(DATA_PATH + "dailyCalories_merged.csv")
daily_intensities = pd.read_csv(DATA_PATH + "dailyIntensities_merged.csv")
sleep = pd.read_csv(DATA_PATH + "sleepDay_merged.csv")

In [10]:
print(daily_activity.shape)
print(sleep.shape)
sleep.head()

(940, 15)
(413, 5)


,Id,SleepDay,TotalSleepRecords,TotalMinutesAsleep,TotalTimeInBed
0,1503960366,4/12/2016 12:00:00 AM,1,327,346
1,1503960366,4/13/2016 12:00:00 AM,2,384,407
2,1503960366,4/15/2016 12:00:00 AM,1,412,442
3,1503960366,4/16/2016 12:00:00 AM,2,340,367
4,1503960366,4/17/2016 12:00:00 AM,1,700,712


# 3. Приведение дат

In [11]:
daily_activity["ActivityDate"] = pd.to_datetime(daily_activity["ActivityDate"]).dt.date
daily_steps["ActivityDay"] = pd.to_datetime(daily_steps["ActivityDay"]).dt.date
daily_calories["ActivityDay"] = pd.to_datetime(daily_calories["ActivityDay"]).dt.date
daily_intensities["ActivityDay"] = pd.to_datetime(daily_intensities["ActivityDay"]).dt.date
sleep["SleepDay"] = pd.to_datetime(sleep["SleepDay"]).dt.date

C:\Users\kokontsev\AppData\Local\Temp\ipykernel_8448\1597672128.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  sleep["SleepDay"] = pd.to_datetime(sleep["SleepDay"]).dt.date


# 4. Формирование дневной таблицы активности

In [13]:
activity = daily_activity[[
    "Id",
    "ActivityDate",
    "TotalSteps",
    "TotalDistance",
    "Calories",
    "VeryActiveMinutes",
    "FairlyActiveMinutes",
    "LightlyActiveMinutes",
    "SedentaryMinutes"
]].copy()

In [14]:
activity = activity.rename(columns={
    "Id": "user_id",
    "ActivityDate": "date"
})

# 5. Подготовка таблицы сна

**Выбираем столбцы**

In [16]:
sleep_df = sleep[[
    "Id",
    "SleepDay",
    "TotalSleepRecords",
    "TotalMinutesAsleep",
    "TotalTimeInBed"
]].copy()

**Приводим к формату**

In [17]:
sleep_df = sleep_df.rename(columns={"Id": "user_id", "SleepDay": "date"})

**Добавляем производную целевую переменную**

In [18]:
sleep_df["SleepEfficiency"] = (
    sleep_df["TotalMinutesAsleep"] / sleep_df["TotalTimeInBed"]
)

# 6. Объединение: активность → сон

**ВАЖНО: Сон ночью относится к предыдущему дню активности!!!**

In [25]:
sleep_df["activity_date"] = sleep_df["date"] - pd.Timedelta(days=1)

df = sleep_df.merge(
    activity,
    left_on=["user_id", "date"],
    right_on=["user_id", "date"],
    how="left",
    suffixes=("", "_activity")
)

df = df.rename(columns={"date": "sleep_date"})

print(df.shape)
df.head()

(413, 14)


,user_id,sleep_date,TotalSleepRecords,TotalMinutesAsleep,TotalTimeInBed,SleepEfficiency,activity_date,TotalSteps,TotalDistance,Calories,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes
0,1503960366,2016-04-12,1,327,346,0.945087,2016-04-11,13162,8.50,1985,25,13,328,728
1,1503960366,2016-04-13,2,384,407,0.943489,2016-04-12,10735,6.97,1797,21,19,217,776
2,1503960366,2016-04-15,1,412,442,0.932127,2016-04-14,9762,6.28,1745,29,34,209,726
3,1503960366,2016-04-16,2,340,367,0.926431,2016-04-15,12669,8.16,1863,36,10,221,773
4,1503960366,2016-04-17,1,700,712,0.983146,2016-04-16,9705,6.48,1728,38,20,164,539


# 7. Сохранение результата

In [28]:
df.to_csv(OUTPUT_PATH + "sleep_activity_daily.csv", index=False)

# 8. Итоги ноубука
- единый user_id
- одна строка = одна ночь сна
- целевая переменная SleepEfficiency
- активность за предыдущий день
- сохранён processed-файл